# Homework 3.

Salette Guadalupe Noemi Villalobos A01246619

Erick Hernández Silva A01750170

Israel Sánchez Miranda A01378705

Loading imports and necessary libraries.

In [1]:
import os
import cv2
import numpy as np
import pandas as pd

from skimage.morphology import label
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold

## Machine Learning training and hyper parameter tuning.

## Statistical-based filter selection and Principal Component Analysis.

## Digit Recognition using Machine Learning.

In this section we use a Machine Learning (ML) algorithm to recognize the digits of one member of the team. We created a dataset that consists in $10$ images each one is a page with $224$ digits each that go from $0$ to $9$. Then our dataset $D$ is compund of digits $d_i$ such that $d_i\in\{0,1,2,3,4,5,6,7,8,9\}$. All digits are handwritten by one member of the team so the algorithm may be able to recognize the handwritting of such member. 

To train our model on the dataset we need to extract the features of each digit. The features we are going to extract are: 
1. The Hu moments. This features are scale and rotation invariant but can be sensitive to noise in the image.
2. The Euler number. This feature will help to distinguish digit holes. 

### Functions and considerations.

Firstly, we need to define a function to segment the $224$ digits from the source image. For this we define a function `segment_digits` that receives the path of the image and the output size of the segmented digit (by default we define an output image of $28\times28$). We apply a threshold to the image using a binary Otsu algorithm, this due to the simple nature of the image where we have a white background (the page itself) and a black object (the digit). Then we use the `cv2.findCountours` function to detect and isolate the individual handwritten digits in the image. We feed to this function the previously thresholded image and ask the algorithm to return the outer contours (`RETR_EXTERNAL`) to avoid nested contours of each digit. We store the values of each corner using the `cv2.CHAIN_APPROX_SIMPLE` format to speed up the process without losing significant information about the shape.

We then iterate over the contours found and calculate its bounding box using the `cv2.boundingRect` function and extract the digit from the thresholded image. We resize this image to the `output_size` and append it to the array that stores this images along with the coordinates of its bounding box.

In [40]:
def segment_digits(image_path, output_size=(28, 28)):
  """
  Segments digits from a handwritten digit image.

  Parameters:
  - image_path: Path to the image file.
  - output_size: Tuple indicating the size to resize the digit images.

  Returns:
  - A list of tuples (digit_image, bounding_box)
  """

  # Load image in grayscale
  img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

  # Threshold the image
  _, thresh = cv2.threshold(img, 255, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
  # Find contours of the digits
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  digit_images = []
  for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    digit = thresh[y:y+h, x:x+w]

    # Resize to standard size
    digit_resized = cv2.resize(digit, output_size, interpolation=cv2.INTER_AREA)
    digit_images.append((digit_resized, (x, y, w, h)))
  return digit_images

Now we define the function `get_features` which extracts the Hu moments and Euler number of a given image. To get the Hu moments the function uses the `cv2.moments` function and then it uses the `cv2.HuMoments` function to finally flatten the result. We make a log scale transformation to reduce the dynamic range of the moments; in this way we improve numerical stability and enchance the discriminative power of the features. 

To calculate the Euler number we threshold the image using the `cv2.THRESH_BINARY` threshold. We then get this number calculating the connected components and substracting the number of holes each digit has (he substract one always to take into account the backgorund that is labeled with `0`).

Finally, we store both the Hu moments and the Euler number in a list.

In [3]:
def get_features(image):
    """
    Calculates Hu moments and Euler number for a given image.

    Parameters:
    - image: The input image.

    Returns:
    - features: A list containing Hu moments and Euler number.
    """

    # Calculate Hu Moments
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments).flatten()

    # Log scale transformation
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)

    # Calculate Euler number
    _, binary_image = cv2.threshold(image, 0, 1, cv2.THRESH_BINARY)
    euler_number = cv2.connectedComponents(binary_image.astype(np.uint8))[0] - 1

    features = list(hu_moments) + [euler_number]
    return features

Another function is defined; the `prepare_dataset`function takes the images from a folder with the following structure: 
- `folder/`
  - `label_0.jpeg`
  - `label_1.jpeg`
  - `label_2.jpeg`
  - `...`
  - `label_n.jpeg

  The function gets the label for each image and then applies the `segment_digits` to extract the digits of each image. Then, for each digit it gets the Hu moments and Euler number using the `get_features` function to finally store all these information into a dataframe.

In [9]:
def prepare_dataset(image_folder):
    """
    Prepares the dataset from images in a folder.

    Parameters:
    - image_folder: Folder containing images (one image per class, class name must be the name of the image).

    Returns:
    - DataFrame containing features and labels.
    """

    data = []
    labels = []
    for filename in os.listdir(image_folder):
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            digit_label = os.path.splitext(filename)[0]
            image_path = os.path.join(image_folder, filename)

            digit_images = segment_digits(image_path)
            for digit_image, _ in digit_images:
                features = get_features(digit_image)
                data.append(features)
                labels.append(digit_label)

    columns = [f'Hu{i+1}' for i in range(7)] + ['EulerNumber']
    df = pd.DataFrame(data, columns=columns)
    df['Digit'] = labels
    return df

We used a $k$-Nearest Neighbors ($k$ NN) algorithm to recognize the digits. We define as default $k=3$. In this function we use the `Digit` column as the target value and use the `KNeighborsClassifier` class from `sklearn` for the $k$ NN algorithm. We used a $k$-Fold cross-validation with $10$ folds using a shuffle in our dataset to better train the model and have a more general overview of how it performs. We print the cross-validation scores and the mean score for each fold. 

In [5]:
def train_knn_model(df, k=3):
    """
    Trains a k-NN classifier using 10-fold cross-validation.

    Parameters:
    - df: DataFrame containing features and labels.
    - k: Number of neighbors for k-NN.

    Returns:
    - Trained k-NN model.
    - Cross-validation scores.
    """

    X = df.drop('Digit', axis=1)
    y = df['Digit']

    knn = KNeighborsClassifier(n_neighbors=k)
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_scores = cross_val_score(knn, X, y, cv=kf)

    knn.fit(X, y)
    print(f'Cross-validation scores: {cv_scores}')
    print(f'Mean CV score: {cv_scores.mean()}')
    return knn, cv_scores

To test our ML algorithm we use the `classify_id_image` function. This function receives the path to an image which is the one containing the ID of a student written by the same member of the team that wrote the digits originally in order for the ML algorithm to recognize them. It firstly segments each digit found on the image using the `segment_digits`function and then for each digit found it gets its features. The function then organizes each feature in a dataframe and makes a prediction with the mdoel using the `predict` function of the $k$ NN model. Finally we sort the predicted digits based on their $x$ coordinate to mantain order, finally, the function returns the digit predictions. 

In [8]:
def classify_id_image(image_path, knn_model):
    """
    Classifies digits from an image containing the student ID.

    Parameters:
    - image_path: Path to the student ID image.
    - knn_model: Trained k-NN model.

    Returns:
    - List of predicted digits in order.
    """

    digit_images = segment_digits(image_path)

    digit_positions = []
    digit_features = []
    for digit_image, bbox in digit_images:
        features = get_features(digit_image)
        digit_features.append(features)
        digit_positions.append(bbox)

    # Predict digits
    X_new = pd.DataFrame(digit_features, columns=[f'Hu{i+1}' for i in range(7)] + ['EulerNumber'])
    predicted_digits = knn_model.predict(X_new)
    
    # Sort digits based on x-coordinate to maintain order
    digits_with_positions = list(zip(predicted_digits, digit_positions))
    digits_with_positions.sort(key=lambda x: x[1][0])  # Sort by x-coordinate
    ordered_digits = [digit for digit, _ in digits_with_positions]
    return ordered_digits


Another way we calculate the performance of the model is using the Levenshtein distance. This is, calculate how many deletions, insertions and substitutions we need to apply over a string $s_j$ to match an initial string $s_i$. This helps us to appreciate how similar is the predicted ID by the model with the original one.

In [67]:
def levenshtein_distance(s1, s2):
    """
    Calculates the Levenshtein distance between two strings.
    """
    
    if len(s1) < len(s2):
      return levenshtein_distance(s2, s1)
    
    # Initialize previous row of distances
    previous_row = list(range(len(s2) + 1))
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            # Cost of deletions, insertions, and substitutions
            deletions = previous_row[j + 1] + 1
            insertions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(deletions, insertions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

### Testing the algorihtm.

We tested the algorithm using thedataset defined at the [start of the section](#digit-recognition-using-machine-learning) using the `prepare_dataset` function over a folder with the structure defined at [Functions and considerations](#functions-and-considerations).

In [48]:
# Prepare the dataset
dataset_folder = 'digits'
df = prepare_dataset(dataset_folder)
print('Dataset prepared.')
df

Dataset prepared.


,Hu1,Hu2,Hu3,Hu4,Hu5,Hu6,Hu7,EulerNumber,Digit
0,3.139321,7.866246,9.607722,9.994978,-10.0,-9.999999,-10.0,1,0
1,2.470154,5.771757,9.556362,9.572124,10.0,-9.999607,-10.0,1,0
2,2.531530,6.019593,9.456941,9.087261,10.0,9.996963,10.0,1,0
3,2.438113,5.863366,9.253229,9.288303,10.0,9.998040,10.0,1,0
4,2.492358,5.891919,9.609553,9.323692,-10.0,9.998321,10.0,1,0
...,...,...,...,...,...,...,...,...,...
2669,2.680575,6.608726,8.188343,9.077046,-10.0,-9.998576,10.0,1,9
2670,2.618261,6.389477,7.991319,8.819356,10.0,-9.998544,10.0,1,9
2671,2.733390,7.603683,8.433619,9.446449,-10.0,9.999985,10.0,1,9
2672,2.676523,6.789756,8.161121,8.907812,10.0,-9.999589,10.0,1,9


We train our $k$ NN model using $k=2$. The model has a score of approximately $0.5$ which is not the best (basically a coin flip) but for educational purposes it works fine.

In [65]:
# Train the k-NN model
k = 2
knn_model, _ = train_knn_model(df, k=k)
print('k-NN model trained.')

Cross-validation scores: [0.57462687 0.55223881 0.45522388 0.54477612 0.52059925 0.4906367
 0.50187266 0.52434457 0.59550562 0.53558052]
Mean CV score: 0.5295404997484487
k-NN model trained.


We then read the ID handwritten image to test the algorithm.

In [66]:
# Classify digits from the student ID image
id_image_path = 'test.jpeg'
predicted_id = classify_id_image(id_image_path, knn_model)
print(f'Predicted Student ID: {"".join(map(str, predicted_id))}, Real ID is 1378705')

Predicted Student ID: 1378730, Real ID is 1378705


As we can see the algorithm works fairly fine on predicting and extracting the ID from the image. We apply the Levenshtein distance function to see how different are both strings. We also apply a normalized error rate to have a better overview of the proportion of the error.

In [68]:
# Calculate Levenshtein distance
predicted_id_str = "".join(map(str, predicted_id))
actual_id_str = '1378705'
error = levenshtein_distance(predicted_id_str, actual_id_str)
print(f'Levenshtein Distance (Error): {error}')

# Normalize Error Rate
max_length = max(len(predicted_id_str), len(actual_id_str))
error_rate = error / max_length
print(f'Normalized Error Rate: {error_rate:.2f}')

Levenshtein Distance (Error): 2
Normalized Error Rate: 0.29


The Levenshtein distance is of $2$, which means that at least two deletions, insertions or substitutions need to be applied for the predicted ID to be the same as the original ID. In this case we need two substitutions,change $s_6=3$ to $s_6=0$ and $s_7=0$ to $s_7=5$. The same way, we have an error rate of $0.29$ which suggest that almost a $30\%$ of the predicted ID is wrong. A way we can improve this algorithm is to do an exploratory search of the $k$ NN hyper parameters (number of neighbors and similar configurations) to achieve an optimal solution. We can also use different threshold values when doing the digit segmentation to avoid extracting digits incorrectly and confuse the model with this input images. Finally, another thing we can do to improve the algorithm is to increase the number of digit samples to have a bigger dataset. For this we would also try to balance more each digit class because in some cases a digit can be written in more than one way. This was not taken into account when handwritting the dataset and may have result in an inter-class imbalance and thus, impacting greatly on the model performance.